In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile
import os
import pandas as pd

# Path to the zip file
zip_file_path = '/content/drive/MyDrive/environment recognition/places365_contents.zip'

# Directory to extract the contents
extracted_dir = 'data_contents/'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Create a list to store dataframe rows
data = []

# Traverse through the extracted directory to create the dataframe
for root, dirs, files in os.walk(extracted_dir):
    for file in files:
        if file.endswith('.jpg'):
            file_path = os.path.join(root, file)
            label_name = os.path.basename(root)
            data.append({'path': file_path, 'name': file, 'label': label_name})

# Create a DataFrame
df = pd.DataFrame(data)

# Adjust path format for 'path name' column
df['path name'] = 'data_contents/' + df['label'] + '/' + df['name']

# Rearrange columns
df = df[['path name', 'name', 'label']]

# Print the DataFrame
print(df)

                                      path name          name          label
0      data_contents/hospital_room/00004975.jpg  00004975.jpg  hospital_room
1      data_contents/hospital_room/00002205.jpg  00002205.jpg  hospital_room
2      data_contents/hospital_room/00002331.jpg  00002331.jpg  hospital_room
3      data_contents/hospital_room/00004407.jpg  00004407.jpg  hospital_room
4      data_contents/hospital_room/00001564.jpg  00001564.jpg  hospital_room
...                                         ...           ...            ...
62515    data_contents/childs_room/00002942.jpg  00002942.jpg    childs_room
62516    data_contents/childs_room/00000819.jpg  00000819.jpg    childs_room
62517    data_contents/childs_room/00003682.jpg  00003682.jpg    childs_room
62518    data_contents/childs_room/00002129.jpg  00002129.jpg    childs_room
62519    data_contents/childs_room/00003051.jpg  00003051.jpg    childs_room

[62520 rows x 3 columns]


In [ ]:
df['label'].value_counts()

airplane_cabin          5000
amusement_arcade        5000
childs_room             5000
bakery-shop             5000
swimming_pool-indoor    5000
ballroom                5000
locker_room             5000
dining_hall             5000
hospital_room           5000
art_gallery             5000
garage-indoor           5000
sauna                   4020
bank_vault              3500
Name: label, dtype: int64

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ReLU, Flatten, Dense
from tensorflow.keras.models import Sequential

def create_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(64, (3, 3), padding='same', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        ReLU(),

        Conv2D(64, (3, 3), padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        ReLU(),

        Conv2D(128, (3, 3), padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        ReLU(),

        Conv2D(128, (3, 3), padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        ReLU(),

        Conv2D(256, (3, 3), padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        ReLU(),

        Conv2D(256, (3, 3), padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        ReLU(),

        Flatten(),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Define input shape and number of classes
input_shape = (112, 112, 3)  # Change this according to your data
num_classes = 13  # Change this to the correct number of classes

# Create the model
model = create_model(input_shape, num_classes)

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 56, 56, 64)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 56, 56, 64)        256       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 56, 56, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 28, 28, 64)        0

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Assuming your DataFrame is named 'df'
# Split the DataFrame into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1.0/255,  # Rescale pixel values to [0, 1]
    rotation_range=20,  # Data augmentation: random rotations
    width_shift_range=0.2,  # Data augmentation: random width shifts
    height_shift_range=0.2,  # Data augmentation: random height shifts
    horizontal_flip=True,  # Data augmentation: horizontal flips
)
batch_size = 32
# Load and preprocess training data from DataFrame
train_generator = datagen.flow_from_dataframe(
    train_df,
    x_col='path name',  # Column containing image paths
    y_col='label',  # Column containing labels
    target_size=(112, 112),  # Resize images to match model input size
    batch_size=batch_size,
    class_mode='categorical',  # Categorical labels for multi-class classification
)

# Load and preprocess validation data from DataFrame
validation_generator = datagen.flow_from_dataframe(
    val_df,
    x_col='path name',
    y_col='label',
    target_size=(112, 112),
    batch_size=batch_size,
    class_mode='categorical',
)
label_map = (train_generator.class_indices)
print(label_map)


Found 50016 validated image filenames belonging to 13 classes.
Found 12504 validated image filenames belonging to 13 classes.
{'airplane_cabin': 0, 'amusement_arcade': 1, 'art_gallery': 2, 'bakery-shop': 3, 'ballroom': 4, 'bank_vault': 5, 'childs_room': 6, 'dining_hall': 7, 'garage-indoor': 8, 'hospital_room': 9, 'locker_room': 10, 'sauna': 11, 'swimming_pool-indoor': 12}


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/environment recognition/model_100_epochs.h5')

In [ ]:
# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(validation_generator)

print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_acc)

# Save the model
#model.save('/content/drive/MyDrive/environment recognition/model_200_epochs.h5')  # Adjust the filename as needed

391/391 [==============================] - 247s 630ms/step - loss: 0.2665 - accuracy: 0.9202
Validation Loss: 0.26654213666915894
Validation Accuracy: 0.9201855659484863
